# 🕵️‍♀️ Zipsa LangGraph Debugger

이 노트북은 LangGraph 에이전트의 구조를 시각화하고, 다양한 질문에 대한 라우팅 및 응답 과정을 단계별로 디버깅하기 위해 제작되었습니다.

In [ ]:
import os
import sys
from dotenv import load_dotenv
from IPython.display import Image, display

# 1. 프로젝트 루트 경로 설정 (src/notebooks -> Project Root)
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "../../"))
sys.path.append(project_root)
sys.path.append(os.path.join(project_root, "src"))

# 2. 환경변수 로드
load_dotenv(os.path.join(project_root, ".env"))

print(f"🔹 Project Root: {project_root}")

In [ ]:
# 3. Graph 로드
from src.agents.graph import create_zipsa_graph
from langchain_core.messages import HumanMessage

app = create_zipsa_graph()

## 📊 Graph Visualization

In [ ]:
try:
    png_data = app.get_graph().draw_mermaid_png()
    display(Image(png_data))
    print("✅ Graph visualization loaded.")
except Exception as e:
    print(f"⚠️ Visualization skipped: {e}")

## 🧪 Interactive Trace
아래 `query` 변수를 변경하여 에이전트의 동작을 테스트하세요.

In [ ]:
# ✅ 테스트 질문 설정
query = "뱅갈 고양이 성격이 궁금해"
# query = "초보 집사인데 키우기 쉬운 고양이 추천해줘"
# query = "고양이가 밥을 안 먹어서 걱정이야"

# Mock Profile
user_profile = {
    "housing": "apartment",
    "activity": "medium",
    "traits": ["순한"]
}

In [ ]:
async def run_trace(query_text: str, profile: dict):
    print(f"❓ Query: {query_text}")
    inputs = {
        "route_history": [],
        "messages": [HumanMessage(content=query_text)],
        "user_profile": profile
    }
    config = {"configurable": {"thread_id": "debug_session"}}
    
    print("Output Trace:")
    async for event in app.astream(inputs, config=config, stream_mode="updates"):
        for node_name, values in event.items():
            print(f"  [NODE]: {node_name}")
            
            if "router_decision" in values:
                print(f"    ↳ Router Decision: {values['router_decision']}")
            
            if "debug_info" in values:
                print(f"    ↳ Debug Info: {values['debug_info']}")
                
            if "messages" in values:
                msgs = values["messages"]
                if msgs:
                    print(f"    ↳ Response: {msgs[-1].content}")

# 실행
await run_trace(query, user_profile)